In [6]:
import bs4
import requests
from urllib import parse
import re
import time
import pandas as pd

def naver_crawling(search_name):

    
    info_lists = []

    session = requests.Session()
    header = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36"}

    print("작품이름 -> ")
    search_name=input()
    tmp = search_name + "리뷰" #혹은 후기

    base_url = "https://search.naver.com/search.naver?date_from=&date_option=0&date_to=&dup_remove=1&nso=&post_blogurl=&post_blogurl_without=blog.me&query="+parse.quote(tmp) + "&sm=tab_pge&srchby=all&st=sim&where=post&start={}"
    n=1
    pre_page=None
    while True:
        url=base_url.format(n)
        r=requests.get(url)
        c=r.content
        html = bs4.BeautifulSoup(c, "html.parser")
        main=html.find('div',{'class','blog section _blogBase _prs_blg'})
        count=main.find('span')
        if(count==None):
            print("프로그램 종료합니다")
            break
        page=count.text.split(" /")[0].replace(" ","")
        if(pre_page==page):
            print("프로그램 종료합니다")
            break
        pre_page=page
        print(page)
        n=n+10
        ul=main.find('ul')
        li_s=ul.findAll('li')

        for li in li_s:
            post=''
            href=li.find('a').attrs['href']
            if ('blog.naver.com' or 'blog.me') not in href:
                continue
            if 'blog.me' in href:
                part_1=href.split('.')[0]
                part_2=href.split('/')[1]
                url="https://m.blog.naver.com/" + part_1 + "/" + part_2
            else:
                url=href.replace("://","://m.")
            print(url)
            req = session.get(url,headers=header)
            html = bs4.BeautifulSoup(req.text, "html.parser")

            body=html.find('div',{'class','_postView'})
            if body!=None:
                post_area=body.find('div',{'class','post_tit_area'})
                if post_area!=None:
                    title=post_area.find('h3').text
                    if search_name.replace(" ","") not in title.replace(" ",""):
                        continue
                    print(title)

                    info_area=post_area.find('div',{'class','author_area'})
                    author_area=info_area.find('div',{'class','se_author'})
                    author=author_area.find('a').find('strong').text
                    print(author)

                    date=info_area.find('p').text
                    print(date)

                    content_area=body.find('div',{'class','post_ct'})
                    contents=content_area.findAll('p')
                    for content in contents:
                        content=content.text
                        post=post+content
                        print(content)                  

                else:
                    post_area=body.find('div',{'class','post_ct'})
                    title_area=post_area.find('div',{'class','se-viewer se-theme-default'})
                    if title_area!=None:
                        info_area=title_area.find('div',{'class','se-component-content'})
                        title=info_area.find('div',{'class','se-module se-module-text se-title-text'}).find('p').text
                        if search_name.replace(" ","") not in title.replace(" ",""):
                            continue
                        print(title)

                        author_area=info_area.find('div',{'class','blog_authorArea'})
                        author=author_area.find('div',{'class','blog_author'}).find('a').find('strong').text
                        print(author)
                        date=author_area.find('p').text
                        print(date)

                        content_area=title_area.find('div',{'class','se-main-container'})
                        contents=content_area.findAll('p')
                        for content in contents:
                            content=content.text
                            post=post+content
                            print(content)                   
                    else:
                        title_area=post_area.findAll('div',{'class','se_component_wrap'})[0]
                        title=title_area.find('h3').text
                        if search_name.replace(" ","") not in title.replace(" ",""):
                            continue
                        print(title)

                        info_area=title_area.find('div',{'class','blog_authorArea'})
                        author_area=info_area.find('div',{'class','blog_author'})
                        author=author_area.find('a').text
                        print(author)

                        date=info_area.find('p').text
                        print(date)

                        content_area=post_area.findAll('div',{'class','se_component_wrap'})[1]
                        contents=content_area.findAll('p')
                        for content in contents:
                            content=content.text
                            post=post+content
                            print(content)
            else:
                body=html.find('div',{'class','postListBody'})
                post_area=body.find('div',{'class','se-viewer se-theme-default'})
                title_area=post_area.find('div',{'class','se-component se-documentTitle se-l-default'})
                title=title_area.find('div',{'class','se-module se-module-text se-title-text'}).find('p').text
                if search_name.replace(" ","") not in title.replace(" ",""):
                    continue
                print(title)

                author_area=title_area.find('div',{'class','blog2_container'})
                author=author_area.find('span').find('a').text
                print(author)

                date=author_area.find('span',{'class','se_publishDate pcol2'}).text
                print(date)

                content_area=post_area.find('div',{'class','se-main-container'})
                contents=content_area.findAll('p')
                for content in contents:
                    content=content.text
                    post=post+content
                    print(content)
                    

            post=re.sub(r'[^ ㄱ-ㅎ|ㅏ-ㅣ|가-힣a-zA-Z0-9\:\/\-\+\*\~\!\`\'\"\!\@\#\$\%\^\&\*\(\.\,\<\/\?\:\;\[\]\{\}\\\|]', '', post)

            info_list = [title, author, date, post]

            info_lists.append(info_list)

    
    df = pd.DataFrame(info_lists, columns = ["title", "author", "date", "post"])

    # csv 파일 만들기
    df.to_csv("{}.csv".format(search_name), index=False, encoding='cp949')
    print("저장 완료.\n")        

In [7]:
naver_crawling('닥터최태수')

작품이름 -> 
닥터최태수
1-10
https://m.blog.naver.com/vetjin72?Redirect=Log&logNo=221985359452
[카카오페이지 소설 리뷰/추천] 4. 닥터 최태수
짠돌이
2020. 5. 31. 20:32
	
​
장르
현판/메디컬
별점
🌕🌕🌕🌕🌑
솔직리뷰
제가 드디어 이 작품을 리뷰하게 되네요ㅋㅋㅋㅋㅋ 웹소설에 관심 없는 사람이라도 이 작품 이름은 들어봤다는 전설의  '닥터 최태수'!!!!!
이 소설 읽기전에, 글 회차가 3000화가 넘어서 약간의 긴장감?과 함께 보기 시작했습니다. 그런데 진짜 재밌더라구요ㅋㅋㅋㅋㅋㅋㅋ 진짜 거짓말 안하고 1000화 읽는데 3일 밖에 안걸렸습니다ㅋㅋㅋㅋㅋ 그렇다고 한 편당 쪽수가 적은것도 아니구요. 그리구 지금은 이 작품이 제 인생작입니다 진짜ㅠㅠㅠㅠ 최태수 쌤이 너무 멋있어요ㅠㅠㅠㅠㅠ
이 소설의 장점이라 하면 주인공이 한없이 산하다는거?? 그리구 실력이 뒷받침이 된가는거ㅋㅋㅋㅋㅋ 음...설명하기가 어렵지만 주인공은 초반부터 먼치킨이 될 실력을 가지고 있지만 점점 성장하면서 완전한 먼치킨이 되는거....? 주인공이 성장하는거 보면 재밌더라구요 저는ㅋㅋㅋㅋㅋㅋ 그리고 제일 중요한!!! 소위 말하는 고구마 부분이 재밌다는겈ㅋㅋㅋㅋㅅㅋㅋㅋ 다른 작품들에서는 고구마 부분 나올것 같다 하면 저는 뛰면서 봅니다...근데 이 작품은 오히려 고구마 부분이 제일 재밌더라고요!!
그리고 제일 중요한 단점.....
1000화 이후로는 점점 질질 끈다는게 느껴집니다.....특히 태수가 전역하고 서울 긴급대응센터에 들어가서 화이트 엔젤 팀을 이끌고 나서 부터 노잼의 길을 걷기 시작합니다ㅏ.... 화이트 엔젤 이후로는 계속 똑같은 내용만 반복하고, 소위 말하는 공장에서 찍어낸 작품....? 그 정도로 똑같은 내용의 반복입니다....
그래두 이 작품 보고 의학 지식이 조금 늘어난 것 같네욬ㅋㅋㅋㅋㅋㅋ 어디가서 아는척 할 수 있을 정도??? 그럼 여기서 리뷰 끝내겠습니다!!
닥터 최태수 추천합니다!!!!
​
​
​



닥터 최태수, 추천, 요약 및 리뷰

코아
2018. 11. 13. 19:59
	

안녕하세요! 신팽입니다^^오늘 소설 리뷰는 [닥터 최태수]라는 퓨전? 현대 판타지? 소설입니다!닥터 최태수는 천재 흉부외과의사 최태수의 스토리입니다.작가는 조석호 작가님으로 현재도 연재중인 인기 소설이죠!워낙 인기가 좋은 작품이다 보니 2015년부터 쭉~ 상위권 웹소설로 등극하고 있고 웹툰으로도 만들어졌습니다.특히 의학드라마를 통해서 의사들의 삶에 대해 어느정도 보신분들이 많겠지만 실제로 병원에서 사용하는 수술 도구들은 일반 사람들이 생각하는 것 보다 엄청나게 다양하고 쓰이는 활용범위 또한 여러종류이죠. 하지만 소설로 제작하면서 많은 독자들이 어려워 할 수 있는 의학 용어, 수술 도구를 쉽게 풀어서 스토리를 진행하다보니 처음 소설에 입문 할 때 그리 어렵지 않았습니다.흉부외과 인턴의사 최태수는 최고를 자부했던 유명 흉부외과 의사의 지식이 머리속으로 들어오게 되면서 최태수라는 사람의 인생에 대한 목표와 삶의 방향이 바뀌게 됩니다. 이런 부분이 가끔은 고구마를 먹여주지만 그래도 소설이니 가볍게 생각하면 입문하는데 큰 장벽이 되진 않을 겁니다^^;;
삶과 죽음이 교차한 순간! 평범한 인턴의 운명이 송두리째 바뀌었다.'2015년 최고의 선택' 이라고 찬사를 받을 정도였고 약 3000화를 앞두고 있는 장편 소설이죠. 미완결!1화부터 정주행을 시작하시면 몇날 몇일이 걸릴지는 모르겠지만 저는 신규 업데이트를 매주 기다리면서 보고 있는 저에게는 기다림이 너무 힘들지경입니다ㅠㅠ그리고 닥터 최태수는 웹툰으로도 제작이 되었습니다.웹툰은 너무 기대는 안하시는게 좋긴하지만 나름 재미가 있더군요. 아무래도 방대한 스토리와 세세한 설명이 웹소설에서는 가능하겠지만 웹툰은 분량에 한계가 있죠. 그리는 것도 나름 시간과 노력을 많이 잡아먹기도 하죠.뭐..웹툰이 드라마로 방영되는걸 보신분들은 어느정도 이해가 되실 듯 하네요.그리고 판타지 적인 내용보다는 흉부외과의 현실을 보여주기 때문에 인간관계와 여러가지 스


외과의로서 환자를 살리기위해 최선을 다하는 태수였지만 열악한 산속에서 장비하나 없이 내장이 파열된 이를 살리는 것은 역부족이었다.

외국 노인의 정체는 바로 세계적으로 권위있는 외과의사인 카프레네.

죽음이 임박한 그는, 태수에게 의사가 윤리적 지침으로 삼는 히포크라테스 선서를 읇도록 시킨 후 자신의 반지를 맡긴다.

그의 죽음을 지켜본 그 날부터, 태수의 꿈엔 자꾸 수술실의 광경이 보이기 시작했다.

머릿속으로 주입되는 의학 지식. 1인칭 시점으로 보여지는 수백 번의 수술 집도.

이 과정을 거친 최태수는 과연 어떤 의사가 될까? 

세계적으로 유명한 이의 모든 기술을 물려받았다.

그도 결국 세계적인 의사가 될 것이다. 그런 그는 과연 명예의 노예, 돈을 위해 의학기술을 파는 장사꾼이 되지않고 끝내 환자를 위한 의사로 남을 수 있을까?

책을 읽고나면, 내가 살아가는 이 세상에도 최태수와 같은 의사가 있기를 간절히 소망하는 당신의 모습을 찾아볼 수 있을것이다.



아래의 주소에서 1권까지 무료로 감상할 수 있다.

http://www.joara.com/premium_new/book_intro.html?book_code=988326


 
https://m.blog.naver.com/ttkdlzh0808?Redirect=Log&logNo=220664291288
[3월작품리뷰] 닥터 최태수_in 조아라/웹소설
은요야
2016. 3. 30. 16:47
	
(제목 이미지 출처 : https://pixabay.com/photo-1137137/)
사실 소개하지 않아도 알 만한 사람들은 이미 아는 작품, '닥터 최태수'입니다.
작품 간략 소개
​
작품명 : 닥터 최태수
장르 : 퓨전
작가명 : 조석호
내용소개 : 
"그래. 환자를...... 무서워해야 돼. 두려워하고 손발이...... 떨리는 공포도 느껴야지."
"......"
"거기서 도망치지...... 않아야 진짜 의사가 되는 거야."
"말씀하시면 출혈이 계속돼요."
반항적인 태수의 말투에도 노인은 미소를 잃지 


닥터최태수-현실보다 더 현실같은,인간 다운.

PaulJ
2017. 4. 21. 13:13
	
닥터최태수조석호
현재 1700화를 향해 가고 있는, 장르소설의 명작 '닥터최태수'리뷰. 닥터최태수 책 장르는 판타지로 구분되어 있지만, 무슨 치유계 마법으로 사람을 손쉽게 고치고 그런 류의 이야기가 아니다. 어쩌면 지극히 현실적이고 인간적인 우리네 이야기를, 삶과 죽음의 경계 최전선에서 싸우는 의사들의 삶을 통해 그려내고 있는 소설이다. 비슷한 소설로 '그레이트써전' 이라는 소설이 있지만, 개인적으로는 닥터최태수가 더 와닿는 소설이라고 말하고 싶다. 그저 무의미한 장르소설이 아니기 때문이다.       닥터최태수를 집필하는 조석호 작가는 현재 만 3년 째 연재를 해오고 있는데, 길어보이지만 사실 1700화라는 작품 수를 볼 때 이건 실로 놀라운 연재속도다. 쉬지도 않고 끊임없이 글을 써서 이야기를 창조해내는 조석호 저자의 능력은 참 경이로울 정도다. 1000일로 계산해도 하루 꾸준히 1.5편이 넘는 분량을 뽑아냈다는 말이 된다. 작가의 성실성은 이미 검증된 바이며, 또 그렇게 분량이 많다고 하여 질질 끌면서 쓸데없는 외전으로 독자들을 지치게 만드는 스토리가 아닌지라 전체적인 호평이 여타 소설들에 비해 상대적으로 굉장히 많은 작품이다. 아마 의학이라는 테마와, 장르소설이라는 테두리 안에서라면 경쟁자가 없으리라 생각한다. 닥터최태수텍본 txt 는 아무래도 온라인 판권 저작권 강화가 심해지다 보니 구할 수 없는 실정이니 깔끔히 포기하기를 권하며, 개인적으론 어서 단행본이 나와서 종이책으로 볼 수 있게 된다면 좋겠다.'닥터최태수'의 이야기는 말 그대로 최태수라는 인턴이 주인공으로 나온다. 가정환경은 열악하고, 그릇된 일을 보면 절대 참지 않고 나서는 의인이 되기를 자처하는 그의 삶은 각박한 세상 가운데 험난하기만 하다. 그러던 어느날 조난당한 등산객을 도와주고자 하나, 이미 최태수의 힘으로는 돌이킬 수 없는 상황이었다. 헌데 등산객은 흉부외과의학계의 세계적 권위자인 '카프


닥터최태수.txt 텍본 7명 (선착순 공유 / 리뷰)

dbsqja134
2016. 4. 14. 17:36
	
 닥터최태수.txt 텍본 7명 (선착순 공유 / 리뷰)
안녕하세요~ ㅎ 오늘 공유를 해드릴 소설은 바로 닥터최태수 라는 소설입니다! 다들 이소설에 대해서 많이 들어보셨을거에요~ ㅎ그래서 제가 여러분에게 공유를 해드리려는 거에요~ ㅎㅎ
닥터최태수.txt공유를 원하시는 분들은 비밀댓글에 이메일 주소를 남겨주시면확인후에 바로 보내드릴게요~ ㅎ 공감도 같이 눌러주세요~ ㅎ
https://m.blog.naver.com/satos?Redirect=Log&logNo=220754946024
[리뷰]닥터 최태수
뿟쨔응
2016. 7. 6. 11:47
 
 
 
필력 : (4/5)주제 : (3.5/5)캐릭터 : (4/5)스토리 : (3.5/5)

필력이 꽤나 괜찮은 소설이긴 하지만, 닥터 최태수의 가장 큰 장점은 현장감이 녹아있는 소설이라는 것이다. 실제로 작가가 의사인지 현장에 대한 서술과 묘사가 몹시 뛰어나다. 의학 관련해서는 '썩션' 이라는 말 밖에 모르는 사람도 이 소설을 읽으면 최소한의 의학 용어를 알게 될지도 모른다.

주제는 의학이다. 사실 최태수라는 인물이 우연히 세계적인 명의 닥터 카프레네의 지식을 얻었다는 것만 제외하면 판타지적인 요소가 거의 없다.
그렇기에 더욱 가치가 있는 소설이다.

캐릭터성이 몹시 뛰어나다. 주조연 인물들의 매력이 뛰어나며 그들의 성격과 특징들로 하여금 캐릭터들이 생생하게 살아있는 것처럼 느끼게 해준다.
또한 환자로 등장하는 일회용 엑스트라의 등장마저도 깊이가 있다.

전체적인 스토리는 최태수가 산에서 사고를 당해 죽어가는 카프레네의 마지막 순간을 옆에서 지켜주게 되고, 그를 계기로 우연히 카프레네의 지식을 얻게 되어 의학을 갈고 닦아나가는 내용이다. 그러나 지식에 의존하지 않고 스스로 갈고 닦아 나간다. 특히 이 소설에서 자세하게 다루는 것은 최태수의 실력보다는 그로 인한 최태수와 다른 의사(혹은 병원)들 간의 갈등이다.
갈

연재_의료) 닥터 최태수
부엉
2017. 10. 17. 22:29
판타지 소설부터, 무협 , 희귀 , 판로 등등등  중학교부터 미친듯이 보고 읽기 시작한소설들을
이곳에 정리할생각으로 씁니다.
 
여태 수많은 소설들을 읽어왔지만, 모든 소설을 제치고, 먼저 소개하고 싶은 소설은
조아라에서 시작했던 소설 "닥터 최태수"를 소개합니다.
 
이번에 드라마제작까지 확정되었지만.....원체 탄탄한 내용전개가 되어있어....
솔직히 카카오페이지에서 연재하는 웹툰도, 뭔가 맞지않아 보지 못하고 있는
저로서는 두근두근 하면서도 기대가 되어 큰일입니다.(보다 실망하면 안되는데....)
 
다만 닥터 최태수 드라마 분들께   로맨스는 있는듯 없이...그냥 느낌만....제발!!!
아니면 그냥 로맨스 빼주세요!!!!!!!!!!!! 라고 할 정도에요. 
 
모든 소설이 그렇듯이 맞는 사람있고, 없는 사람 있지만. 읽어서 후회는 안하는 소설이라고
저는 추천 드립니다. ㅎㅎㅎ
 
돈도 아깝지 않습니다.
 
 
닥터 최태수 

리뷰보기

​
https://m.blog.naver.com/elf703?Redirect=Log&logNo=220501342743
닥터 최태수 
elf703
2015. 10. 6. 19:13
닥터 최태수  (조아라 연재중/저자:조석호/~415회차 연재중)


(사진은 관계없는 드라마 켑쳐/어쩐지 소설과 어울리는듯 하여^^)

개인적으로 취미에 맞지않는 현판소설이라 
처음 읽기에 망설여진 작품이랍니다
특히 의학소설류는 예전부터 드라마에 자주 등장하던 소재라 
판타지관련 취미인 본인으로선 그닥 접하진 않은 부류랄까....... (일본만화원작 "진"같은것 취향)

하지만 평범한 인턴이 다른이의(외국 대빵 유명한 의사)기억을 전이받는 과정부터....
뭔가 읽어볼까란 느낌으로 다가왔달까
아직까지 의료분야 소설중에 사실적으로 이렇게까지 다룬 것은 없는듯 하여(인터넷 연재소설중)....

그리고
.......................재미있다란 느낌으로 해외파견(지원)대목까지 읽

https://m.blog.naver.com/milkgreat?Redirect=Log&logNo=222019496578
https://m.blog.naver.com/sangsan?Redirect=Log&logNo=221279054026
https://m.blog.naver.com/aktmt1109?Redirect=Log&logNo=221578489814
https://m.blog.naver.com/psl3629?Redirect=Log&logNo=221969722558
https://m.blog.naver.com/ttkdlzh0808?Redirect=Log&logNo=220813147750
81-90
https://m.blog.naver.com/kjhs2831?Redirect=Log&logNo=221972282421
https://m.blog.naver.com/kakao_page?Redirect=Log&logNo=220930630638
https://m.blog.naver.com/radin21?Redirect=Log&logNo=221703815040
https://m.blog.naver.com/wnaks1543?Redirect=Log&logNo=221019314605
https://m.blog.naver.com/guswlsdl122?Redirect=Log&logNo=221306678115
https://m.blog.naver.com/nesde?Redirect=Log&logNo=221843320317
https://m.blog.naver.com/jjs645?Redirect=Log&logNo=221026281166
https://m.blog.naver.com/dudalschl123?Redirect=Log&logNo=220794738911
https://m.blog.naver.com/lunatickid?Redirect=Log&logNo=220669585198
https://m.blog.naver.com/aktmt1109?Redirect=Log&logNo=2217514